In [1]:
#--------- Python Libraries ---------#

import requests
import json
import re
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import csv
import pandas as pd
from datetime import date
import datetime
import dateutil.parser

import SpotifyLookup


import string
import Levenshtein as lev




In [2]:
#--------- Web crawling for URLs ---------#

source = requests.get('https://devilution.dk/anmeldelser/koncertanmeldelser').text
soup = BeautifulSoup(source, 'lxml')
print(source)


<!DOCTYPE html>
<html lang="da-dk" dir="ltr">
<head>
		<script type="text/javascript" src="//ajax.googleapis.com/ajax/libs/jquery/1.8.3/jquery.min.js"></script>
	<script type="text/javascript">jQuery.noConflict();</script>

			<base href="https://devilution.dk/anmeldelser/koncertanmeldelser" />
	<meta http-equiv="content-type" content="text/html; charset=utf-8" />
	<meta name="keywords" content="Devilution, heavy metal, heavyjam.dk, dødsmetal, thrash, metallica, volbeat, anmeldelser, pladeanmeldelser, koncertanmeldelser, dansk metal," />
	<meta name="description" content="Devilution - webmagasin om hård rock og heavy metal" />
	<meta name="generator" content="Joomla! - Open Source Content Management" />
	<title>Koncertanmeldelser</title>
	<link href="/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon" />
	<link href="/plugins/system/jce/css/content.css?a6c1507275b5d464ee860c44a83fc5e3" rel="stylesheet" type="text/css" />
	<link href="/modules/mod_fewestread/fewestread.cs

In [3]:
list_of_URLs = []

# Loop through all review links to find concert reviews specifically
for link in soup.find_all('div',{'class' : 'jrListingTitle'}):
    test = link.find('a')
    URL = test.get('href')
    URL = 'https://devilution.dk' + URL 
    list_of_URLs.append(URL)
    print('Found concert review URL')

# Remove duplicate URLs in list
list_of_URLs = list(set(list_of_URLs))
print(list_of_URLs)

Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
Found concert review URL
['https://devilution.dk/anmeldelser/koncertanmeldelser/vltimas-1349-goeteborg-2020', 'https://devilution.dk/anmeldelser/koncertanmeldelser/five-finger-death-punch-f8', 'https://devilution.dk/anmeldelser/koncertanmeldelser/memoriam-paletten-060320', 'https://devilution.dk/anmeldelser/koncertanmeldelser/decapitated-voxhall-08032020', 'https://devilution.dk/anmeldelser/koncertanmeldelser/extremely-rotten-death-metal-byhaven-17072020', 'https://devilution.dk/anmeldelser/koncertanmeldelser/petrol-gir

In [4]:
# 2 test URLs
#list_of_URLs = ['https://devilution.dk/anmeldelser/koncertanmeldelser/baest-studenterhuset-aalborg-2020']

# Create empty dictionary
critic_review_dict = {} 
critic_review_dict['concerts'] = []

# Loop - For every link in the URL_links do the following with the reviews:
for i in list_of_URLs:

    # Reset values
    critic_name = 'Devilution' 
    author = ''
    url = ''
    headline = ''
    text = ''
    rating = 0
    max_rating = 5
    tribute_rating = 0
    artist_name = ''
    venue_name = ''
    venue_place = ''
    publication_date = ''
    concert_date = ''
    lang = 'da'
    genre = []
    
    url = i
    
    # Get the source code of the webpage as text
    source = requests.get(i).text
    soup = BeautifulSoup(source, 'lxml')
    
    ######### Clean Soup for correct text mining #########
    
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    
    for div in soup.find_all("div", {'class':'oo-line'}): 
        div.decompose()
        
    
    # Find headline
    try:
        headline = soup.h1.text
        headline = headline.lstrip()
        print('Headline found')
    
    except:
        print('No headline found')
    
    
    # Find text
    try:
        text = soup.find('div',{'class' : 'jrListingFulltext'})

        text_list = []

        for i in text.find_all('p'):
            text_list.append(i.get_text())

        text = text_list

        text = '\n\n'.join(text)
        print('Text found')

    except:
        print('No text found')
    
    
    
    # Find artist name
    try:
        artist_name = soup.find('div', {'class': 'jrFieldRow jrArtist'})
        artist_name = artist_name.find('div', {'class': 'jrFieldValue'}).getText()
        artist_name = artist_name.lstrip()
        print('artist name found')
        print(artist_name)

    except:
        artist_name = ''
        print('No artist name found')
        continue
    
    # Find venue name
    try:
        venue_name = soup.find('div', {'class': 'jrFieldRow jrSpillested'})
        venue_name = venue_name.find('div', {'class': 'jrFieldValue'}).getText()
        venue_name = venue_name.lstrip()
        print('venue name found')

    except:
        print('No venue name found')
    
    # Find concert date
    try:
        concert_date = soup.find('div', {'class': 'jrFieldRow jrDato'})
        concert_date = concert_date.find('div', {'class': 'jrFieldValue'}).getText()
        
        # Format date
        concert_date = concert_date.lstrip()
        default_time = 'T00:00:00+01:00'
        concert_date = concert_date+default_time
        concert_date = dateutil.parser.parse(concert_date, dayfirst=True)
        print('Concert date found')

    except:
        print('No concert date found')
        continue
    
    # Find publication date
    try:    
        publication_date = soup.find('time', {'itemprop' : 'datePublished'})
        publication_date = publication_date["datetime"]
        publication_date = dateutil.parser.parse(publication_date)
        print('publication date found')

    except:
        publication_date = ''
        print('No publication_date found') 
        continue
    
    # Find Author
    try:
        author = soup.find('div', {'class': 'jrFieldRow jrAuthor'})
        author = author.find('div', {'class': 'jrFieldValue'}).getText()
        author = author.lstrip()
        print('Author found')

    except:
        print('No author found')
    
    # Find rating
    try:
        rating = soup.find('div', {'class': 'jrFieldRow jrKarakter'})
        rating = rating.find('div', {'class': 'jrFieldValue'})
        rating = rating.find('img')
        rating = int(rating['title'])
        tribute_rating = (5/max_rating)*rating
        tribute_rating = round(tribute_rating, 2)
        print('Rating found')

    except:
        print('No rating found')
    
    # Find Genre
    try:
        genre_value = soup.find('div', {'class': 'jrFieldRow jrGenre'})
        genre_value = genre_value.find('div', {'class': 'jrFieldValue'}).getText()
        genre_value = genre_value.lstrip()
        genre.append(genre_value)
        print('Genre found')
    except:
        print('No genre found')
    
    # Spotify Lookup with artist_name
    spotify_results = SpotifyLookup.spotify_lookup(artist_name)
    artist_name_spotify = spotify_results[0]
    artist_img_spotify = spotify_results[1]
    artist_spotify_id = spotify_results[2]
    subgenres_spotify = spotify_results[3]
    artist_popularity_spotify = spotify_results[4]
    
    print('')
    print('Critic:', critic_name)
    print('Author:', author)
    print('URL:', url)
    print('Artist:', artist_name)
    print('Venue:', venue_name)
    print('Venue Place:', venue_place)
    print('Rating:', str(rating))
    print('Max Rating:', str(max_rating))
    print('Tribute Rating:', str(tribute_rating))
    print('Publication Date:', publication_date)
    print('Concert Date:', concert_date)
    print('Genre:', genre)
    print('artist spotify name:', artist_name_spotify)
    print('artist spotify image:', artist_img_spotify)
    print('artist spotify ID:', artist_spotify_id)
    print('artist spotify subgenres:', subgenres_spotify)
    print('artist spotify popularity:', artist_popularity_spotify)
    print('Language of review:', lang)
    print('Headline:', headline)
    print('Text:', text)
    
    # Set dictionary values

    temp_dict = {}
    temp_dict['critic_name'] = critic_name
    temp_dict['author'] = author
    temp_dict['url'] = url
    temp_dict['artist_name'] = artist_name
    temp_dict['artist_name_spotify'] = artist_name_spotify
    temp_dict['artist_img_spotify'] = artist_img_spotify
    temp_dict['artist_spotify_id'] = artist_spotify_id
    temp_dict['artist_popularity_spotify'] = artist_popularity_spotify
    temp_dict['genre'] = genre
    temp_dict['subgenre_spotify'] = subgenres_spotify
    temp_dict['publication_date'] = publication_date
    temp_dict['concert_date'] = concert_date
    temp_dict['venue_name'] = venue_name
    temp_dict['venue_place'] = venue_place
    temp_dict['rating'] = rating
    temp_dict['tribute_rating'] = tribute_rating
    temp_dict['max_rating'] = max_rating
    temp_dict['review_language'] = lang
    temp_dict['headline'] = headline
    temp_dict['text'] = text

    critic_review_dict['concerts'].append(temp_dict)

    
    
    


Headline found
Text found
artist name found
Vltimas
No venue name found
Concert date found
publication date found
Author found
Rating found
No genre found
Name preprocessing: Vltimas = vltimas
found 1 results
searching for exact string match...
Found exact string match on index 0 VLTIMAS

Critic: Devilution
Author: Kent Kirkegaard Jensen
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/vltimas-1349-goeteborg-2020
Artist: Vltimas
Venue: None
Venue Place: 
Rating: 3
Max Rating: 5
Tribute Rating: 3.0
Publication Date: 2020-02-16 15:47:18+01:00
Concert Date: 2020-02-14 00:00:00+01:00
Genre: []
artist spotify name: VLTIMAS
artist spotify image: https://i.scdn.co/image/ab67616d0000b273b473618c57913051dc2a1d49
artist spotify ID: 0kZ64zYv2xtKdhbOzTPdWA
artist spotify subgenres: ['black death']
artist spotify popularity: 25
Language of review: da
Headline: Fokus på Vincent og Frost

Text: De to opvarmningsbands på Trädgår'n i Göteborg denne aften, Vltimas og 1349, havde hver sin person

Headline found
Text found
artist name found
Memoriam
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Memoriam = memoriam
found 8 results
searching for exact string match...
Found exact string match on index 0 Memoriam

Critic: Devilution
Author: Svend Lokjær
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/memoriam-paletten-060320
Artist: Memoriam
Venue: Paletten
Venue Place: 
Rating: 3
Max Rating: 5
Tribute Rating: 3.0
Publication Date: 2020-03-07 12:27:12+01:00
Concert Date: 2020-03-06 00:00:00+01:00
Genre: ['Dødsmetal']
artist spotify name: Memoriam
artist spotify image: https://i.scdn.co/image/9bb68eb370edd9287a071a1569cfb0e2800e768e
artist spotify ID: 2EAy7g3Ga8hS10hAHMf6C4
artist spotify subgenres: ['birmingham metal', 'death metal']
artist spotify popularity: 26
Language of review: da
Headline: VMF '20: Et hyggeligt og halvtræt gensyn

Text: Memoriams velmente og stereotype nostalgidød var et par oprig

Headline found
Text found
artist name found
Undergang
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Undergang = undergang
found 6 results
searching for exact string match...
Found exact string match on index 0 Undergang

Critic: Devilution
Author: Jacobh Hansen
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/extremely-rotten-death-metal-byhaven-17072020
Artist: Undergang
Venue: Pumpehuset
Venue Place: 
Rating: 3
Max Rating: 5
Tribute Rating: 3.0
Publication Date: 2020-07-19 17:04:35+02:00
Concert Date: 2020-07-17 00:00:00+01:00
Genre: ['Dødsmetal']
artist spotify name: Undergang
artist spotify image: https://i.scdn.co/image/ab67616d0000b27353248b5180dc68af42a17552
artist spotify ID: 1fFtjgqU6Z2vKBmk5Z8z3o
artist spotify subgenres: ['black death', 'cavernous death metal', 'danish death metal', 'danish metal', 'voidgaze', 'voidgrind']
artist spotify popularity: 25
Language of review: da
Headline: Død på bænk

Headline found
Text found
artist name found
Konkhra
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Konkhra = konkhra
found 2 results
searching for exact string match...
Found exact string match on index 0 Konkhra

Critic: Devilution
Author: Svend Lokjær
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/konkhra-atlas-210220
Artist: Konkhra
Venue: Atlas
Venue Place: 
Rating: 4
Max Rating: 5
Tribute Rating: 4.0
Publication Date: 2020-02-22 09:52:04+01:00
Concert Date: 2020-02-21 00:00:00+01:00
Genre: ['Dødsmetal']
artist spotify name: Konkhra
artist spotify image: https://i.scdn.co/image/ab67616d0000b273864ba6c0285c255c0d9a2aae
artist spotify ID: 17TLTIgidZF0ypQJRBpp60
artist spotify subgenres: ['danish death metal']
artist spotify popularity: 11
Language of review: da
Headline: Dansk dødsnostalgi

Text: Konkhra serverede velsmagende dødsmetal fra encyklopædiens første sider, mens begge supportbands trods publik

Headline found
Text found
artist name found
Alcest
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Alcest = alcest
found 3 results
searching for exact string match...
Found exact string match on index 0 Alcest

Critic: Devilution
Author: Tobias Holst
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/alcest-pumpehuset-200215
Artist: Alcest
Venue: Pumpehuset
Venue Place: 
Rating: 2
Max Rating: 5
Tribute Rating: 2.0
Publication Date: 2020-02-16 13:07:51+01:00
Concert Date: 2020-02-15 00:00:00+01:00
Genre: ['Progressiv']
artist spotify name: Alcest
artist spotify image: https://i.scdn.co/image/4c106ec022e645089b4bbf5942312964a3f21171
artist spotify ID: 0d5ZwMtCer8dQdOPAgWhe7
artist spotify subgenres: ['atmospheric black metal', 'blackgaze', 'french black metal', 'french metal', 'french shoegaze', 'post-black metal', 'post-metal', 'shoegaze']
artist spotify popularity: 51
Language of review: da
Headline: Blackgaze 

...

Headline found
Text found
artist name found
Hamferð
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Hamferð = hamferð
found 1 results
searching for exact string match...
Found exact string match on index 0 Hamferð

Critic: Devilution
Author: Malthe Yde Tiufkær
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/nocturnefest-2020
Artist: Hamferð
Venue: Loppen
Venue Place: 
Rating: 4
Max Rating: 5
Tribute Rating: 4.0
Publication Date: 2020-03-03 11:43:23+01:00
Concert Date: 2020-02-29 00:00:00+01:00
Genre: ['Doom']
artist spotify name: Hamferð
artist spotify image: https://i.scdn.co/image/8277dd1b301f67e0358e5715095d599dc014afb4
artist spotify ID: 4Okrb4EaAYXpRu2cx2ipa4
artist spotify subgenres: ['doom metal', 'faroese rock', 'funeral doom', 'opera metal']
artist spotify popularity: 21
Language of review: da
Headline: Drager og Genfærd

Text: På en røgfyldt aften med drager og genfærd forvandlede Loppen sig for 3.

Headline found
Text found
artist name found
Sum 41
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Sum 41 = sum 41
found 1 results
searching for exact string match...
Found exact string match on index 0 Sum 41

Critic: Devilution
Author: Hanna Ella Sandvik
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/sum-41-amager-bio-030220
Artist: Sum 41
Venue: Amager Bio
Venue Place: 
Rating: 3
Max Rating: 5
Tribute Rating: 3.0
Publication Date: 2020-02-04 10:56:18+01:00
Concert Date: 2020-02-03 00:00:00+01:00
Genre: ['Punk']
artist spotify name: Sum 41
artist spotify image: https://i.scdn.co/image/3ad9fb703dffe649bec3ec92abccbb280276316b
artist spotify ID: 0qT79UgT5tY4yudH9VfsdT
artist spotify subgenres: ['alternative metal', 'canadian pop punk', 'canadian punk', 'nu metal', 'pop punk', 'punk']
artist spotify popularity: 75
Language of review: da
Headline: Fanget i fortiden

Text: Et taknemmeligt publikum og et velspi

Headline found
Text found
artist name found
Natjager
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Natjager = natjager
found 1 results
searching for exact string match...
Found exact string match on index 0 Natjager

Critic: Devilution
Author: Gabriel Thordahl-Jensen
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/natjager-150220-beta
Artist: Natjager
Venue: Beta
Venue Place: 
Rating: 4
Max Rating: 5
Tribute Rating: 4.0
Publication Date: 2020-02-17 06:39:35+01:00
Concert Date: 2020-02-15 00:00:00+01:00
Genre: ['Nu metal']
artist spotify name: Natjager
artist spotify image: https://i.scdn.co/image/d67108d14d6f6b72d2ab4cf60c0270235ab6659b
artist spotify ID: 1HLapTU4X0HOlgGbGYRlNB
artist spotify subgenres: ['danish metal']
artist spotify popularity: 9
Language of review: da
Headline: Masser af kærlighed, masser af glæde

Text: Had dem eller elsk dem, Natjager har ikke kun formået at spille på både Copenhell o

Headline found
Text found
artist name found
Abbath
No venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Abbath = abbath
found 3 results
searching for exact string match...
Found exact string match on index 0 Abbath

Critic: Devilution
Author: Kent Kirkegaard Jensen
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/abbath-goeteborg-2020
Artist: Abbath
Venue: None
Venue Place: 
Rating: 4
Max Rating: 5
Tribute Rating: 4.0
Publication Date: 2020-02-16 15:15:50+01:00
Concert Date: 2020-02-14 00:00:00+01:00
Genre: ['Black Metal']
artist spotify name: Abbath
artist spotify image: https://i.scdn.co/image/7cde591a10a6a85ca851546cebbd9d8d9c0029f8
artist spotify ID: 1epGwdbjU7JSGVBHlqptpx
artist spotify subgenres: ["black 'n' roll", 'black metal', 'norwegian black metal', 'symphonic black metal']
artist spotify popularity: 35
Language of review: da
Headline: Godt comeback til Abbath

Text: Den tidligere frontmand i Immorta

Headline found
Text found
artist name found
Steen Jørgensen
venue name found
Concert date found
publication date found
Author found
Rating found
Genre found
Name preprocessing: Steen Jørgensen = steen jørgensen
found 2 results
searching for exact string match...
No exact match on result from index  0 Steen Chorchendorff Jorgensen
Found exact string match on index 1 Steen Jørgensen

Critic: Devilution
Author: Jacobh Hansen
URL: https://devilution.dk/anmeldelser/koncertanmeldelser/steen-jorgensen-dr-koncerthus-26-06-20
Artist: Steen Jørgensen
Venue: DR Koncerthus
Venue Place: 
Rating: 4
Max Rating: 5
Tribute Rating: 4.0
Publication Date: 2020-06-26 23:14:48+02:00
Concert Date: 2020-06-26 00:00:00+01:00
Genre: ['Rock']
artist spotify name: Steen Jørgensen
artist spotify image: https://i.scdn.co/image/ab67616d0000b273739db9d99840a5d115249096
artist spotify ID: 2lnlfVDzgudEJ6dPgkBHEd
artist spotify subgenres: ['danish singer-songwriter']
artist spotify popularity: 12
Language of review: da
